<a href="https://colab.research.google.com/github/itovvmk/itovvmk.github.io/blob/main/Language_and_Health_Pain_in_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atualização e importação de módulos

In [1]:
# Usar arquivos no drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Instalação/atualização do spaCy e dos modelos de língua
!pip install -U spacy

!python -m spacy download es_core_news_lg

2024-01-01 18:00:02.457119: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 18:00:02.457198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 18:00:02.469207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 18:00:02.503382: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 18:00:08.097038: W tensorflow/compiler/tf2

In [3]:
from sklearn.manifold import TSNE
import numpy as np

In [4]:
!pip install gensim==3.8

In [5]:
import os

In [6]:
import gensim #solve import errors as in https://stackoverflow.com/questions/69381312/importerror-cannot-import-name-mapping-from-collections-using-python-3-10
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import pandas as pd

In [7]:
# Importação do módulo e do modelo de língua
import spacy
nlp = spacy.load('es_core_news_lg')
import pandas as pd
from nltk.corpus import PlaintextCorpusReader
import nltk
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import itertools

def achatar(lista):
    return list(itertools.chain(*lista))

In [271]:
# Stopwords
stops_nltk = nltk.corpus.stopwords.words('spanish')
novas_stops = ['fondo', 'dávila','e', 'cachai', 'cachay', 'poh','eh', 'po', 'ah', 'pa', 'así', 'ay'
               'em','m','xxx', 'inaud', 'aah', 'risas','súper','entretenido', 'venezuela', 'antofagasta', 'er', 'él','yo',
               'entonces', 'si', 'sí', 'pucha', 'puta',
               'quizás', 'ser', 'sé', 'igual','cha', 'pu', 'allí',
               'ahí', 'claro', 'va', 'bueno','forma','alguna','doctorado',
               'ejemplo', 'hecho','quizá', 'tal', 'realidad', 'entrecomillas', 'súper',
               'comillas', 'indisa', 'antropología', 'antropologa','sociales','chiloé','acá'
               'tema', 'r', 'l','s','h','bonito', 'años', 'año','cosa','cosas','d','copita']
stops_nltk.extend(novas_stops)
stops_spacy = nlp.Defaults.stop_words
stops = set(stops_nltk) | stops_spacy

In [10]:
# Pegar arquivos .txt da pasta no drive
from nltk.corpus import PlaintextCorpusReader

corpus_root = '/content/drive/My Drive/RiCL/entrevistas'
wordlists = PlaintextCorpusReader(corpus_root, '.*')

# Criação da coleção de documentos
entrevistas = wordlists.fileids()

corpus = list()

In [11]:
import re

In [12]:
""" Função que recebe uma string e lematiza
"""
def preproc(sent):
    tokens = nltk.tokenize.word_tokenize(sent)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    tokens = [i for i in tokens if i not in stops]
    return tokens

def lema(string):
    doc = nlp(string)

    return [token.lemma_ for token in doc]

In [272]:
# Transformar em lista de strings
corpus = []
for entrevista in entrevistas:
    strings = wordlists.raw(entrevista)
    corpus.append(strings)

# Retirar falas de entrevistadoras
regex = r"M*:.*?E:"

colecao_entrevistadas = []
for i in corpus:
    achar_regex = re.findall(regex, i, re.DOTALL)
    entrevistadas = ' '.join(achar_regex)
    colecao_entrevistadas.append(entrevistadas)

In [66]:
len(colecao_entrevistadas)

30

In [273]:
tokens = []

In [274]:
# Limpar e tokenizar
tokens = [preproc(i) for i in colecao_entrevistadas]

In [275]:
tokens_strings = [' '.join(i) for i in tokens]

In [276]:
lemas = [lema(i) for i in tokens_strings]

# Word2Vec implementation

In [277]:
# WORD2VEC for corpus in Spanish
from gensim.models import Word2Vec
# size: dimensão dos vetores
# window: distância máxima (em palavras) a partir da palavra-alvo
# min_count: número mínimo de ocorrências das palavras escolhidas

model = Word2Vec(lemas, size=120, window=10, min_count=10)

# total_examples: número de sentenças de treinamento
model.train(lemas, total_examples=30, epochs=100)

(1971155, 3632100)

# Most similar

# Definitions of menstruation

In [278]:
# termos similares
model.most_similar('menstruación', topn=10)

<ipython-input-278-9ebf28e0e23b>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('menstruación', topn=10)


[('abundante', 0.6191399693489075),
 ('hueso', 0.5212849378585815),
 ('hemorragia', 0.46438682079315186),
 ('doloroso', 0.4471602439880371),
 ('cortar', 0.42838358879089355),
 ('anticonceptivo', 0.41756123304367065),
 ('tremendo', 0.4049299359321594),
 ('durar', 0.3983049690723419),
 ('severo', 0.3864632546901703),
 ('sangrar', 0.3787758946418762)]

In [279]:
# termos similares
model.most_similar('regla', topn=10)

<ipython-input-279-22afe12de5db>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('regla', topn=10)


[('abundante', 0.49064308404922485),
 ('empezar', 0.43338245153427124),
 ('desmayar', 0.39458146691322327),
 ('mes', 0.38473016023635864),
 ('doloroso', 0.37647801637649536),
 ('oportunidad', 0.37431246042251587),
 ('sabi', 0.3717849850654602),
 ('invalidante', 0.3545975983142853),
 ('durar', 0.3523443043231964),
 ('típico', 0.35159099102020264)]

In [280]:
# termos similares
model.most_similar('menstrual', topn=10)

<ipython-input-280-8d7fd94c6881>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('menstrual', topn=10)


[('abundante', 0.5358511209487915),
 ('molestia', 0.49564170837402344),
 ('adolescencia', 0.4488529562950134),
 ('abdomen', 0.42090243101119995),
 ('estómago', 0.39602816104888916),
 ('pélvico', 0.3956192433834076),
 ('regular', 0.39012280106544495),
 ('aliviar', 0.3870812654495239),
 ('analgésico', 0.37223589420318604),
 ('durar', 0.37035876512527466)]

# Definitions of endometriosis

In [281]:
model.most_similar('endometriosis', topn=10)

<ipython-input-281-89715920f301>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('endometriosis', topn=10)


[('especialista', 0.5998467206954956),
 ('investigar', 0.4279727637767792),
 ('profundo', 0.4089559316635132),
 ('sospechar', 0.3975067734718323),
 ('enterar', 0.3711698651313782),
 ('resonancia', 0.369948148727417),
 ('común', 0.3680271506309509),
 ('hacer él', 0.35261496901512146),
 ('grado', 0.3495524525642395),
 ('información', 0.33816033601760864)]

# Definitions of pain

In [282]:
model.most_similar('dolor', topn=10)

<ipython-input-282-0a8877908de0>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('dolor', topn=10)


[('constante', 0.5843042731285095),
 ('contracción', 0.5290500521659851),
 ('espalda', 0.4617069363594055),
 ('abdomen', 0.45693260431289673),
 ('intenso', 0.4491477906703949),
 ('hueso', 0.43398958444595337),
 ('analgésico', 0.43102115392684937),
 ('abundante', 0.42961353063583374),
 ('niña', 0.4243719279766083),
 ('molestia', 0.41971537470817566)]

In [283]:
model.most_similar('doler', topn=10)

<ipython-input-283-d14c8c2ecf25>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('doler', topn=10)


[('reglar', 0.4419650435447693),
 ('durar', 0.42462119460105896),
 ('normal', 0.40757301449775696),
 ('espalda', 0.40254896879196167),
 ('gana', 0.3937356472015381),
 ('dolor', 0.35173511505126953),
 ('descansar', 0.3349992632865906),
 ('regla', 0.33118951320648193),
 ('olvidar', 0.31992366909980774),
 ('guata', 0.3161216974258423)]

In [292]:
model.most_similar('doloroso', topn=10)

<ipython-input-292-8a34660e33a7>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('doloroso', topn=10)


[('recuerdo', 0.5962249040603638),
 ('recordar', 0.5763002634048462),
 ('abundante', 0.5587742328643799),
 ('menstruación', 0.44716018438339233),
 ('periodo', 0.39578723907470703),
 ('diagnosticar', 0.3822478652000427),
 ('regla', 0.37647804617881775),
 ('adolescencia', 0.3743680715560913),
 ('tremendo', 0.37252724170684814),
 ('cortar', 0.347270131111145)]

In [284]:
model.most_similar('impacto', topn=10)

<ipython-input-284-bc02554901a7>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('impacto', topn=10)


[('negativo', 0.6616787910461426),
 ('social', 0.5915586352348328),
 ('económico', 0.5901157855987549),
 ('efecto', 0.5320744514465332),
 ('lugar', 0.5113654136657715),
 ('condición', 0.5073619484901428),
 ('laboral', 0.5066367983818054),
 ('reunión', 0.5003066062927246),
 ('acompañar', 0.45932653546333313),
 ('emocional', 0.45082616806030273)]

In [285]:
model.most_similar('adolescencia', topn=10)

<ipython-input-285-3c9e687a3b79>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('adolescencia', topn=10)


[('ibuprofeno', 0.5109801888465881),
 ('niña', 0.48325273394584656),
 ('abundante', 0.46499890089035034),
 ('dolo', 0.45540884137153625),
 ('menstrual', 0.4488529562950134),
 ('recuerdo', 0.436631977558136),
 ('regular', 0.43432775139808655),
 ('periodo', 0.43388593196868896),
 ('abdomen', 0.3903155028820038),
 ('doloroso', 0.3743680715560913)]

In [286]:
model.most_similar('colegio', topn=10)

<ipython-input-286-5d01d521eba6>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('colegio', topn=10)


[('universidad', 0.45253342390060425),
 ('paracetamol', 0.44880425930023193),
 ('tení', 0.4262995719909668),
 ('sentado', 0.42406609654426575),
 ('completo', 0.40740835666656494),
 ('urgencia', 0.37605828046798706),
 ('recordar', 0.34494563937187195),
 ('desmayé', 0.34488001465797424),
 ('físico', 0.3409912586212158),
 ('amigo', 0.32751262187957764)]

In [288]:
model.most_similar('apoyo', topn=10)

<ipython-input-288-18d08d83ca7a>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('apoyo', topn=10)


[('principal', 0.5770919919013977),
 ('apoyado', 0.49126067757606506),
 ('aportar', 0.47231757640838623),
 ('parejo', 0.4694792628288269),
 ('apoyar', 0.4540311098098755),
 ('cercano', 0.4534933269023895),
 ('familiar', 0.44950729608535767),
 ('red', 0.4242996275424957),
 ('ayuda', 0.3952217102050781),
 ('salud', 0.39272838830947876)]